In [24]:
import numpy as np

In [25]:
x_value = [3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9]
y_value = [-0.00011, 0.543966, 1.051358, 1.469572, 1.753617, 1.870829, 1.804553, 1.556275, 1.145949, 0.610438]

In [26]:
# Вычислим конечные разности
delta_y = np.diff(y_value)

# Экстраполяция
y0_extrapolated = y_value[0] - delta_y[0]
yN_extrapolated = y_value[-1] - delta_y[-1]
print(f"Экстраполированное значение в начале: {y0_extrapolated}")
print(f"Экстраполированное значение в конце: {yN_extrapolated}")

Экстраполированное значение в начале: -0.5441860000000001
Экстраполированное значение в конце: 1.145949


In [28]:
def lagrange_interpolation(x, x_value, y_value):
  n = len(x_value)
  result = 0
  for i in range(n):
    term = y_value[i]
    for j in range(n):
      if j != i:
        term *= (x - x_value[j])/(x_value[i] - x_value[j])
    result += term
  return result

# Интреполяция для полушагов
x_half_steps = [3.05, 3.15, 3.25, 3.35, 3.45, 3.55, 3.65, 3.75, 3.85]
y_interpolated = [lagrange_interpolation(x, x_value, y_value) for x in x_half_steps]
print(f"Значения функции в полушагах: {y_interpolated}")

Значения функции в полушагах: [0.27308651704406794, 0.8056420980682354, 1.2746609579010015, 1.6307459119720462, 1.8345266233673092, 1.8609723450164797, 1.7023408493804935, 1.3694294206390394, 0.8909572843780484]


In [29]:
def divided_differences(x, y):
  n = len(y)
  coef = list(y)
  for j in range(1, n):
    for i in range(n-1, j-1, -1):
      coef[i] = (coef[i] - coef[i-1]) / (x[i] - x[i-j])
    return coef

def newton_poly(x, dataX, coef):
  n = len(dataX) - 1
  p = coef[n]
  for k in range(1, n+1):
    p = coef[n-k] + (x - dataX[n - k])*p
  return p

In [30]:
# Значение близкое к центру
x_center = x_value[len(x_value) // 2]

# Интерполирование методом Ньютона
coef = divided_differences(x_value, y_value)
y_newton = newton_poly(x_center, x_value, coef)

print(f"f({x_center}) = {y_newton}")

f(3.5) = 4.021474343999997


In [31]:
# Берем точку рядом с центром таблицы
i_center = len(x_value) // 2
x_center = x_value[i_center]

# Вычислим h, преполагая, что интервалы +- одинаковые
h = x_value[1] - x_value[0]

# Производная первого порядка с использованием центральной разности
f_prime = (y_value[i_center + 1] - y_value[i_center - 1]) / (2*h)

# Производная второго порядка с использованием центральной разности
f_double_prime = (y_value[i_center + 1] - 2*(y_value[i_center]) + y_value[i_center - 1]) / (2*h)

print(f"f'({x_center}) = {f_prime}")
print(f"f''({x_center}) = {f_double_prime}")

f'(3.5) = 0.25468000000000024
f''(3.5) = -0.9174399999999997


In [32]:
import numpy as np
# Выберем 6 узлов данных из таблицы
x_value = [3.0, 3.1, 3.2, 3.3, 3.4, 3.5]
y_value = [-0.00011, 0.543966, 1.051358, 1.469572, 1.753617, 1.870829]

n = len(x_value) - 1  # number of subintervals

# Составим систему уравнений
A = np.zeros((2*n, 2*n))
b = np.zeros(2*n)

for i in range(n):
    # Уравнения для значений функции
    A[i, 2*i:2*(i+1)] = x_value[i]**2, x_value[i]
    b[i] = y_value[i]

    # Уравнения непрерывности функций
    if i != n-1:
        A[n+i, 2*i:2*(i+2)] = x_value[i+1]**2, x_value[i+1], -x_value[i+1]**2, -x_value[i+1]
        b[n+i] = y_value[i+1] - y_value[i]

# Доп. условие: установка первой производной в ноль в конечных точказ
A[-2, 0:2] = 2 * x_value[0], 1
b[-2] = 0
A[-1, -2:] = 2 * x_value[-1], 1
b[-1] = 0

# Поиск коэффициентов сплайна
coeffs = np.linalg.solve(A, b)

# Определим функцию сплайна
def spline(x):
    for i in range(n):
        if x_value[i] <= x < x_value[i+1] or (i == n-1 and x == x_value[-1]):
            return coeffs[2*i]*x**2 + coeffs[2*i+1]*x
    return None

# Оценка для промежуточных узлов
intermediate_x = [(x_value[i] + x_value[i+1]) / 2 for i in range(n)]
intermediate_y = [spline(x) for x in intermediate_x]

print("Промежуточные значения:")
for x, y in zip(intermediate_x, intermediate_y):
    print(f"x = {x:.2f}, y = {y:.5f}")

Промежуточные значения:
x = 3.05, y = 0.53519
x = 3.15, y = 1.04357
x = 3.25, y = 1.46348
x = 3.35, y = -3.57799
x = 3.45, y = 1.75469


In [35]:
# Примеры данных из таблицы (от 0 до 2π будем использовать выбранные значения)
x_value = [3.0, 3.1, 3.2, 3.3, 3.4, 3.5]
y_value = [-0.00011, 0.543966, 1.051358, 1.469572, 1.753617, 1.870829]

# Предположим, что узлы расположены на одинаковом расстоянии друг от друга, вычислим h
h = x_value[1] - x_value[0]

# Коэффициенты Фурье с использованием правила трапеции
def compute_coefficients():
    a0 = (1/(2*np.pi)) * sum(y_value) * h

    ak = []
    bk = []
    for k in range(1, 4):
        ak.append((1/np.pi) * sum([y_value[i] * np.cos(k * x_value[i]) for i in range(len(x_value))]) * h)
        bk.append((1/np.pi) * sum([y_value[i] * np.sin(k * x_value[i]) for i in range(len(x_value))]) * h)
    return a0, ak, bk

a0, ak, bk = compute_coefficients()

# Тригонометрический полином
def trig_polynomial(x):
    result = a0/2
    for k in range(3):
        result += ak[k] * np.cos((k+1)*x) + bk[k] * np.sin((k+1)*x)
    return result

# Проверка полинома на отрезке [0; 2π]
test_x = np.linspace(0, 2*np.pi, 100)
test_y = [trig_polynomial(x) for x in test_x]

# Результаты тестирования
for x, y in zip(test_x, test_y):
    print(f"x = {x:.2f}, P(x) = {y:.5f}")

x = 0.00, P(x) = -0.12495
x = 0.06, P(x) = -0.13739
x = 0.13, P(x) = -0.14569
x = 0.19, P(x) = -0.14948
x = 0.25, P(x) = -0.14861
x = 0.32, P(x) = -0.14309
x = 0.38, P(x) = -0.13319
x = 0.44, P(x) = -0.11933
x = 0.51, P(x) = -0.10216
x = 0.57, P(x) = -0.08243
x = 0.63, P(x) = -0.06104
x = 0.70, P(x) = -0.03898
x = 0.76, P(x) = -0.01726
x = 0.83, P(x) = 0.00309
x = 0.89, P(x) = 0.02110
x = 0.95, P(x) = 0.03585
x = 1.02, P(x) = 0.04660
x = 1.08, P(x) = 0.05272
x = 1.14, P(x) = 0.05379
x = 1.21, P(x) = 0.04959
x = 1.27, P(x) = 0.04016
x = 1.33, P(x) = 0.02572
x = 1.40, P(x) = 0.00677
x = 1.46, P(x) = -0.01603
x = 1.52, P(x) = -0.04176
x = 1.59, P(x) = -0.06938
x = 1.65, P(x) = -0.09770
x = 1.71, P(x) = -0.12544
x = 1.78, P(x) = -0.15126
x = 1.84, P(x) = -0.17384
x = 1.90, P(x) = -0.19190
x = 1.97, P(x) = -0.20427
x = 2.03, P(x) = -0.20991
x = 2.09, P(x) = -0.20797
x = 2.16, P(x) = -0.19784
x = 2.22, P(x) = -0.17914
x = 2.28, P(x) = -0.15176
x = 2.35, P(x) = -0.11589
x = 2.41, P(x) = -0.07

In [37]:
# Примеры данных из таблицы
x_value = [3.0, 3.1, 3.2, 3.3, 3.4, 3.5]
y_value = [-0.00011, 0.543966, 1.051358, 1.469572, 1.753617, 1.870829]

# Вычислим начальное отклонение
y_mean = np.mean(y_value)
initial_std_dev = np.sqrt(np.mean((y_value - y_mean)**2))

for degree in range(1, len(x_value)):  # We can fit up to degree (n-1) for n data points
    # Подбор многочлена заданной степени
    coeffs = np.polyfit(x_value, y_value, degree)
    p = np.poly1d(coeffs)

    # Считаем подобранные значения y
    y_fitted = p(x_value)

    # Считаем среднеквадратичное отклонение
    rmse = np.sqrt(np.mean((y_value - y_fitted)**2))

    # Показатель качества
    quality = rmse / initial_std_dev

    print(f"Степень: {degree}, Качество (Среднеквадр. отклонение/Нач. станд. отклонение): {quality:.5f}")


Степень: 1, Качество (Среднеквадр. отклонение/Нач. станд. отклонение): 0.20501
Степень: 2, Качество (Среднеквадр. отклонение/Нач. станд. отклонение): 0.03586
Степень: 3, Качество (Среднеквадр. отклонение/Нач. станд. отклонение): 0.00230
Степень: 4, Качество (Среднеквадр. отклонение/Нач. станд. отклонение): 0.00019
Степень: 5, Качество (Среднеквадр. отклонение/Нач. станд. отклонение): 0.00000
